In [54]:
# Install Pillow, pytesseract and translate, via Conda or pip
# Install Tesseract on your PC, https://github.com/UB-Mannheim/tesseract/wiki
from pytesseract import pytesseract
from translate import Translator

import re
import string

import cv2
import os

import csv

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\20190735\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Helper functions

In [56]:
# Transcribes image to text
def transcribe(img):
    transcription = pytesseract.image_to_string(img)

    return transcription

In [57]:
# Translate text to given language
def translate(text, lang):
    translator = Translator(to_lang=lang)
    translation = translator.translate(text)

    return translation

# Data Creation

In [58]:
def preprocess(text):
    text = [i.split('\n') for i in text.split(' ')]
    text = [item for sublist in text for item in sublist]
    text = [i for i in text if i not in stopwords.words('english')]
    text = ' '.join(text)
    text = text.lower() 
    text = text.strip()  
    text = re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    text = ' '.join([i for i in dict.fromkeys(text.split(' ')) if len(i) > 1])
    return text

In [59]:
# Transcribe all images in image folder to texts
def transcribe_folder(img_folder, target):
    data = []
    
    for img_name in os.listdir(img_folder):
        img_path = os.path.join(img_folder, img_name)
        
        try:
            img = cv2.imread(img_path)
            text = transcribe(img)
            text = preprocess(text)
            
            if text == "":
                os.remove(img_path)
                continue
            
            text = re.sub(r'\n', ' ', text)
            data.append({'ImageName': img_name, 'Text': text, 'Target': target})
        except TypeError:
            "error"
            
    return data

In [60]:
# Save data in csv file
def save_data(data, csv_file):
    fieldnames = ['ImageName', 'Text', 'Target']
        
    with open(csv_file, 'w', encoding='utf-8') as file:
        file.truncate()
        writer = csv.DictWriter(file, fieldnames=fieldnames, lineterminator = '\n')
        writer.writeheader()
        writer.writerows(data)

In [61]:
# Create data
def create_data(img_folder, target, csv_file):
    data = transcribe_folder(img_folder, target)
    save_data(data, csv_file)

In [62]:
create_data(r'data\imgs_creditcard', 1, r'data\txt_creditcard.csv')

In [63]:
create_data(r'data\imgs_login', 1, r'data\txt_login.csv')